In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 1. Define feature engineering function
def feature_engineering(df):
    # Ensure model_year is integer
    df['model_year'] = df['model_year'].fillna(df['model_year'].median()).astype(int)
    # Vehicle age
    df['vehicle_age'] = 2025 - df['model_year']
    df['vehicle_age'] = df['vehicle_age'].replace(0, 0.5)
    # Flags and codes
    df['drivable_flag'] = (df['drivable'] == 'Y').astype(int)
    df['make_code'] = df['make'].astype('category').cat.codes
    # Interaction features
    df['mileage_age'] = df['mileage'] * df['vehicle_age']
    df['grade_drivable'] = df['grade'] * df['drivable_flag']
    # Buckets
    df['mileage_bucket'] = pd.cut(df['mileage'], bins=[0,50000,100000,150000,200000,300000], labels=False)
    df['age_bucket'] = pd.cut(df['vehicle_age'], bins=[0,3,5,8,12,20], labels=False)
    # Average price by make
    avg_price_make = df.groupby('make')['gross_sale_price'].mean()
    df['make_avg_price'] = df['make'].map(avg_price_make)
    return df

# 2. Load datasets
cars = pd.read_csv('Customer Database.csv', encoding='latin1')
cars_test = pd.read_excel('OH VINs w. Zips.xlsx')

# Add Location to test set
cars_test['Location'] = 'OHIO'

# Drop unused columns (ignore errors if columns missing)
cars = cars.drop(columns=['color','sale_date'], errors='ignore')
cars_test = cars_test.drop(columns=['VIN','Auction lights - Green','Auction lights - Red','Zip','Transport Operable','Transport Inop','Representation','Sold Date'], errors='ignore')

# Rename columns in test set
cars_test = cars_test.rename(columns={
    'Year':'model_year',
    'Make':'make',
    'Model':'model',
    'Mileage':'mileage',
    'Vehicle Grade (Approved CR)':'grade',
    'Operable':'drivable'
})

# Clean grade and drivable in test
cars_test['grade'] = (cars_test['grade']
                        .str.replace('Grade','',regex=False)
                        .str.replace(': Salvage','',regex=False)
                        .str.replace('Unknown','',regex=False)
                        .replace('', np.nan))
cars_test['grade'] = pd.to_numeric(cars_test['grade'], errors='coerce')
cars_test['drivable'] = cars_test['drivable'].replace('Yes','Y')

# Apply feature engineering
cars = feature_engineering(cars)
cars_test = feature_engineering(cars_test)

# 3. Define features and target
numeric_feats = ['mileage','model_year','grade','vehicle_age','drivable_flag','make_code','mileage_age','grade_drivable','mileage_bucket','age_bucket','make_avg_price']
categorical_feats = ['make','model','Location','drivable']
target = 'gross_sale_price'

# Prepare training data
data = cars[numeric_feats + categorical_feats + [target]].dropna()
X = data[numeric_feats + categorical_feats]
y = data[target]

# Split into train/validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Create tf.data datasets
batch_size = 32

def df_to_dataset(features, labels=None, shuffle=True, batch_size=32):
    if labels is not None:
        ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
        if shuffle:
            ds = ds.shuffle(buffer_size=len(features))
    else:
        ds = tf.data.Dataset.from_tensor_slices(dict(features))
    return ds.batch(batch_size)

train_ds = df_to_dataset(X_train, y_train, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(X_val, y_val, shuffle=False, batch_size=batch_size)

# 5. Build feature inputs for TF model
all_inputs = []
encoded_features = []

# Numeric feature inputs + normalization
for feat in numeric_feats:
    inp = tf.keras.Input(shape=(1,), name=feat)
    norm = tf.keras.layers.Normalization(name=f'norm_{feat}')
    norm.adapt(X_train[[feat]])
    x = norm(inp)
    all_inputs.append(inp)
    encoded_features.append(x)

# Categorical feature inputs + one-hot
for feat in categorical_feats:
    inp = tf.keras.Input(shape=(1,), name=feat, dtype='string')
    lookup = tf.keras.layers.StringLookup(output_mode='one_hot', name=f'lookup_{feat}')
    lookup.adapt(X_train[feat])
    x = lookup(inp)
    all_inputs.append(inp)
    encoded_features.append(x)

# Concatenate all features
x = tf.keras.layers.concatenate(encoded_features)

# 6. Define the neural network
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
output = tf.keras.layers.Dense(1, name='price')(x)

model = tf.keras.Model(inputs=all_inputs, outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

# 7. Train the model
epochs = 50
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

# 8. Evaluate
eval_loss, eval_rmse = model.evaluate(val_ds)
print(f"Validation RMSE: {eval_rmse:.2f}")

# 9. Predict on external test set
X_test = cars_test[numeric_feats + categorical_feats]
test_ds = df_to_dataset(X_test, labels=None, shuffle=False, batch_size=batch_size)
preds = model.predict(test_ds).flatten()

# 10. Save predictions
results = cars_test.copy()
results['DL_Predictions'] = preds
results.to_excel('DL_Price_Predictions.xlsx', index=False)
print('Saved predictions to DL_Price_Predictions.xlsx')


ModuleNotFoundError: No module named 'tensorflow'